In [3]:
from data_read import *
from odometry_functions import *
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.optimize import least_squares

In [ ]:
f = 
cx = 
cy  = 
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.4,
                       minDistance = 7,
                       blockSize = 7,
                       useHarrisDetector=False)
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [4]:
data_loader=Data_load(input_line=64, frames=5)

In [5]:
lidar_frames,gt_frames,img_frames=data_loader.read_frames(batch=1, if_removal=False)

In [8]:
features = find_keypoints(img_frames[0,:,:,:],lidar_frames[0,:,:,0],feature_params)
next_features, status, error = track_features(img_frames[0,:,:,:],features,img_frames[1,:,:,:],lk_params)

In [11]:
features = features[status==1]
next_features = next_features[status==1]

In [10]:
compute_3D_points(lidar_frames[0,:,:,:],features,f,cx,cy)

array([[788.3947 , 236.33604],
       [248.28384, 167.99547],
       [687.65356, 212.00035],
       [258.7893 , 193.64713],
       [241.18109, 134.8558 ],
       [295.3255 , 180.9068 ],
       [287.70145, 176.74435],
       [736.1919 , 221.43701]], dtype=float32)